This first part is for working within Google Colab:

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 77.2 MB/s eta 0:00

In [2]:
import os
import torch
import pandas as pd

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/iberlef/IberLEF2023

/content/gdrive/MyDrive/iberlef/IberLEF2023


In [5]:
#! git status

Pretraining PolitiBETO on this year's IberLEF shared task dataset that has been cleaned with [preprocess.ipynb](https://github.com/Abbywpy/IberLEF2023/blob/main/preprocess.ipynb).

## Preprocessing

In [6]:
if not os.path.exists("./data/full_data/cleaned/cleaned_full_data.csv"):
    df = pd.read_csv("./data/full_data/cleaned/cleaned_full_data.csv")
    df["cleaned_tweet"] = df["cleaned_tweet"].astype(str)
    df_tweets = pd.DataFrame(df["cleaned_tweet"])
    df_tweets.to_csv("./data/full_data/cleaned/cleaned_tweets_only.csv", index=False)

In [7]:
from datasets import load_dataset

datafile = "cleaned_tweets_only.csv" # replace the path for according to the data you want to use
dataset = load_dataset('csv', data_files=os.path.join("./data/full_data/cleaned", datafile)).shuffle(seed=42)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8dd713ccab6a0ca7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
""" this is only to test if the code works, don't use in actual training """
#from datasets import DatasetDict

#print(type(dataset))
#dataset = dataset["train"].select(range(1000))

#dataset = DatasetDict({"train": dataset})

#print(type(dataset))

" this is only to test if the code works, don't use in actual training "

In [9]:
print(dataset["train"][0])

{'cleaned_tweet': ':B_button_(blood_type): Agradable E Intensa Mañana de Reuniones Y Visitas Por Bilbao con Elixabete Etxanobe, Diputada Foral de Administración Pública Y Relaciones Institucionales .. :small_orange_diamond:En @ user Hemos Reflexionado Sobre Diversos Temas de Bilbao-Bizkaia, con El Compromiso de Colaboración.'}


In [10]:
from transformers import BertTokenizer

model_checkpoint = "nlp-cimat/politibeto"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)


def tokenize_function(examples):
    return tokenizer(examples["cleaned_tweet"], padding=True, truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/180000 [00:00<?, ? examples/s]

## Model

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # for GPU usage

In [12]:
from transformers import BertForMaskedLM

# if using existing checkpoint
# model_checkpoint = "./pretrained_politibeto/tweets_only"

model = BertForMaskedLM.from_pretrained(model_checkpoint)
model.to(device) # for GPU usage

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ./pretrained_politibeto/tweets_only were not used when initializing BertForMaskedLM: ['distilbert.transformer.layer.11.attention.q_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.9.ffn.lin2.bias', 'distilbert.transformer.layer.8.attention.k_lin.weight', 'distilbert.transformer.layer.7.attention.q_lin.weight', 'distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.3.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.10.ffn.lin1.bias', 'distilbert.transformer.layer.9.ffn.lin1.weight', 'distilbert.transformer.layer.5.sa_layer_norm.weight', 'distilbert.transformer.layer

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [18]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir="./pretrained_politibeto2/" + datafile[:-4].split("_",1)[1],
    overwrite_output_dir=True,
    num_train_epochs=3,  # hyperparameters the same as mentioned in Villa-Cueva et al. (2022)
    per_device_train_batch_size=64, # apart from this --> 128 in original, bc RAM crashes
    learning_rate=5e-5,
    save_total_limit=3 # total checkpoints that are saved
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

In [19]:
# Training (based on PolitiBETO)
trainer.train()

Step,Training Loss
500,6.625300
1000,6.083400
1500,5.856000
2000,5.688200
2500,5.492900
3000,5.250300
3500,5.003500
4000,4.820700
4500,4.648200
5000,4.536800


TrainOutput(global_step=8439, training_loss=4.9429084858293715, metrics={'train_runtime': 13677.462, 'train_samples_per_second': 39.481, 'train_steps_per_second': 0.617, 'total_flos': 3.55328487936e+16, 'train_loss': 4.9429084858293715, 'epoch': 3.0})

In [17]:
# Continue training from the last checkpoint using our dataset
#checkpoint_dir = "./pretrained_politibeto/tweets_only/"
#trainer.train(resume_from_checkpoint=checkpoint_dir)

There were missing keys in the checkpoint model loaded: ['bert.embeddings.position_ids', 'bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias',

OutOfMemoryError: ignored

In [20]:
trainer.save_model()